<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/DB_07_eventlog_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **=== 0. 簡介 ===**
1. Data, dataset, dataloader
2. Customize pre-train model
3. Train model and track result
4. View result
5. Set multiple model experiments
6. Check multiple results at once
7. Load best model and make prediction

# **=== 1. Data, dataset, dataloader ===**


## 1.1 Setups

In [ ]:
!pip install torchinfo

In [ ]:
import matplotlib.pyplot as plt

import torch
import torchvision

from torch import nn
from torchvision import transforms
from torchinfo import summary

## 1.2 Raw data

In [ ]:
import os
import zipfile
from pathlib import Path
import requests

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Directory
if image_path.is_dir():
    print(f"[INFO] {image_path} directory already esixts")
else:
    print(f"[INFO] {image_path} directory not found, creating ...")
    image_path.mkdir(parents = True, exist_ok = True)

    # Download .zip file
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f: # 依照路徑, 創造檔案 -> 此時該檔案是空白檔案
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print(f"[INFO] downloading data")
        f.write(request.content) # 對空白檔案寫入 request 的內容

    # Unzip file
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_f:
        print(f"[INFO] unziping data")
        zip_f.extractall(image_path)

    # Remove .zip file
    zip_file_path = data_path / "pizza_steak_sushi.zip"
    if zip_file_path.exists():
        print(f"[INFO] removing .zip file")
        os.remove(zip_file_path)

[INFO] data/pizza_steak_sushi directory not found, creating ...
[INFO] downloading data
[INFO] unziping data
[INFO] removing .zip file


## 1.3 Transforms

In [11]:
# Train & test directory path
train_dir = image_path / "train"
test_dir = image_path / "test"

# Transfer model's transforms
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
transform = weights.transforms()

## 1.4 Dataset

In [14]:
from torchvision import datasets

train_dataset = datasets.ImageFolder(root = train_dir, transform = transform)
test_dataset = datasets.ImageFolder(root = test_dir, transform = transform)

## 1.5 Dataloader

In [15]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset = train_dataset, batch_size = 10, shuffle = True, drop_last = True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = 10, shuffle = False, drop_last = True)

# **=== 2. Customize pretrained model ===**

## 1.2 Get model

In [32]:
model_1 = torchvision.models.efficientnet_b0(weights = weights)

## 1.3 Freeze feature extraction layer

In [36]:
for param in model_1.features.parameters():
    param.requires_grad = False

## 1.4 Customize classifier layer

In [37]:
model_1.classifier = nn.Sequential(
    nn.Dropout(p = 0.2, inplace = True),
    nn.Linear(in_features = 1280, out_features = 1000, bias = True),
    nn.ReLU(),
    nn.Dropout(p = 0.2, inplace = True),
    nn.Linear(in_features = 1000, out_features = 512, bias = True),
    nn.ReLU(),
    nn.Dropout(p = 0.2, inplace = True),
    nn.Linear(in_features = 512, out_features = 128, bias = True),
    nn.ReLU(),
    nn.Dropout(p = 0.2, inplace = True),
    nn.Linear(in_features = 128, out_features = 16, bias = True),
    nn.ReLU(),
    nn.Dropout(p = 0.2, inplace = True),
    nn.Linear(in_features = 16, out_features = 3, bias = True)
)

## 1.5 Check model info

In [38]:
summary(model_1,
        input_size = (10, 3, 224, 224),
        col_names = ["output_size", "num_params", "trainable"],
        col_width = 15)

Layer (type:depth-idx)                                  Output Shape    Param #         Trainable
EfficientNet                                            [10, 3]         --              Partial
├─Sequential: 1-1                                       [10, 1280, 7, 7] --              False
│    └─Conv2dNormActivation: 2-1                        [10, 32, 112, 112] --              False
│    │    └─Conv2d: 3-1                                 [10, 32, 112, 112] (864)           False
│    │    └─BatchNorm2d: 3-2                            [10, 32, 112, 112] (64)            False
│    │    └─SiLU: 3-3                                   [10, 32, 112, 112] --              --
│    └─Sequential: 2-2                                  [10, 16, 112, 112] --              False
│    │    └─MBConv: 3-4                                 [10, 16, 112, 112] (1,448)         False
│    └─Sequential: 2-3                                  [10, 24, 56, 56] --              False
│    │    └─MBConv: 3-5              

# **=== 3. Train model & check results ===**

## 3.1 Functions for practice

In [ ]:
# return train_loss, train_acc
def train(model, dataloader, loss_fn, optimizer, device):
